# (Main Title)
## (sub title) (Classification/Regression)

**Author**: Miguel Santana

Thank you for reviewing this repository. The author's contact info, blog post, sources and social media profiles are listed below under **further information.**

The contents of this repository detail an analysis of ()

The analysis will provide insight into ()

Lastly, used cases that it can be applied to include ()

#### Project Framework | OSEMN

**Data processing and analysis is completed using the OSEMN framework. The structure includes: Obtaining the data, Scrubbing (processing), Exploratory Data Analysis, Statistical Modeling and Interpretation of the Results.**

#### The Data

The dataset includes ()

The dataset can be found ()

#### Package Imports

In [ ]:
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# Import Data

In [ ]:
# df = pd.read_csv('.csv')

# print(df.shape)
# print('\n')
# df.head(3)

# Scrubbing | Data Cleaning
### Null Values

In [ ]:
# fill process
# df.dropna(inplace=True)
# df.dropna(subset=['feature'],inplace=True)

### Feature Engineering

In [ ]:
# feature engineer

# Exploratory Data Analysis
### Multicollinearity

## Feature Relationship | Feature 1

## Feature Relationship | Feature 2

## Model Preparation
### Separating Features & Target 

In [ ]:
# X = df.drop('target',axis=1)
# y = df['target']

# Statistical Modeling 

```Python
# From sklearn.**model_family** import **ModelAlgo** 

# mymodel = **ModelAlgo**(param1,param2)

# mymodel.fit(x_train,y_train)

# predictions = mymodel.predict(x_test)

# From sklearn.metrics import **error_metric**

# performance = **error_metric**(y_test,predictions)
```

### Model Validation

```Python
# Mean absolute error: mean of, sum of, absolute value of errors
# Mean squared error: mean of, sum of, value of errors-squared  
Root mean squared error: Square root of, mean of, sum of, squared errors
* compare your error metric to average value (shoot for under 10%)
```

```Python
from sklearn.metrics import mean_absolute_error,mean_squared_error

mean_absolute_error(y_test,test_predictions) # MSE / compare to mean

np.sqrt(mean_squared_error(y_test,test_predictions)) # RMSE / compare to mean

df['target'].mean() # for comparison

```

#### Plotting Residuals

```Python
test_residuals = y_test - test_predictions # errors from model

# plot to make sure no clear issue
sns.scatterplot(x=y_test,y=test_residuals) # actual vs test

# plot the horizontal line
plt.axhline(y=0,color='r',ls='--') # should be random/distr around zero

sns.displot(test_residuals,bins=25,kde=True) # check for normal distribution of errors

# normal probability plot / compare perfect normal distr against error values
import scipy as sp

# compare your quantiles to the theoretical ones if they were normal distr
# Create a figure and axis to plot on
fig, ax = plt.subplots(figsize=(6,8),dpi=100)
# probplot returns the raw values if needed
# we just want to see the plot, so we assign these values to _
_ = sp.stats.probplot(test_res,plot=ax) # should be close to the line for distr

```

# Interpreting Results | Recommendations / Used Cases

The model was able to predict () with an (metric) score of (score).

This type of an analysis is popular in business case ()

It allows companies to better ()

#### Feature Selection/Top Features/Recommendations

**(Illustration)**

### Limitations, Future Work

#### Limitations
Limitations to this type of statistical modeling are ()

Challenges for this analysis are ()

#### Future Work

Future work should include () data in order to ()

This will allow () to be able to () better and ultimately leverage statistical modeling in order to ()

#### Further Information

A blog post on this analysis can be found on [Medium.](website)

For any additional questions, please reach out via email at santana2.miguel@gmail.com, on [LinkedIn](https://www.linkedin.com/in/miguel-angel-santana-ii-mba-51467276/) or on [Twitter.](https://twitter.com/msantana_ds)

#### Sources

Additional analysis, notes and file sources can be found on Udemy's website. 

* Course Name:
* Instructor: 